# 第五章 检索(Retrieval)

 - [一、向量数据库检索](#一、向量数据库检索)
     - [1.1 相似性搜索](#1.1-相似性搜索)
     - [1.2 解决多样性：最大边际相关性(MMR)](#1.2-解决多样性：最大边际相关性(MMR))
     - [1.3 解决特殊性：使用元数据](#1.3-解决特殊性：使用元数据)
     - [1.4 解决特殊性：在元数据中使用自查询检索器](#1.4-解决特殊性：在元数据中使用自查询检索器)
     - [1.5 其他技巧：压缩](#1.5-其他技巧：压缩)
 - [二、结合各种技术](#二、结合各种技术)
 - [三、其他类型的检索](#三、其他类型的检索)



检索是我们的检索增强生成(RAG)流程的核心。

让我们获得前面课程存储的向量数据库(`VectorDB`)。

## 一、向量数据库检索

在当前文件夹下新建`.env`文件，内容为`OPENAI_API_KEY = "sk-..."`

本章节需要使用`lark`包，运行以下命令安装

In [1]:
!pip install -Uq lark

In [3]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']


### 1.1 相似性搜索

In [50]:
from langchain.vectorstores import Chroma
# from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = 'docs/chroma/cs229_lectures/'
persist_directory_chinese = 'docs/chroma/matplotlib/'

将上节课所保存的向量数据库(`VectorDB`)加载进来

In [4]:
from dotenv import load_dotenv
env = load_dotenv("/home/xinyi/.env")

import os

from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_core.messages import HumanMessage

llm = ChatTongyi(
    streaming=True,
)

res = llm.stream([HumanMessage(content="hi")], streaming=True)
for r in res:
    print("chat resp:", r.content)
from langchain.embeddings.dashscope import DashScopeEmbeddings
embedding = DashScopeEmbeddings(
            model="text-embedding-v1",
    )

chat resp: Hello
chat resp: !
chat resp:  How
chat resp:  can I assist you today
chat resp: ?


In [6]:
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

In [7]:
print(vectordb._collection.count())

209


In [53]:
# vectordb_chinese.delete_collection()
vectordb_chinese = Chroma(
    persist_directory=persist_directory_chinese,
    embedding_function=embedding
)

In [54]:
print(vectordb_chinese._collection.count())

27


让我们现在来看看最大边际相关性的例子。因此，我们将从下面示例中加载有关蘑菇的信息。

让我们现在运行它与MMR。让我们传入k等于2。我们仍然希望返回两个文档，但让我们设置获取k等于3，其中我们最初获取所有三个文档。我们现在可以看到，我们检索的文档中返回了有毒的信息。

In [10]:
texts = [
    """The Amanita phalloides has a large and imposing epigeous (aboveground) fruiting body (basidiocarp).""",
    """A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.""",
    """A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.""",
]

In [11]:
texts_chinese = [
    """毒鹅膏菌（Amanita phalloides）具有大型且引人注目的地上（epigeous）子实体（basidiocarp）""",
    """一种具有大型子实体的蘑菇是毒鹅膏菌（Amanita phalloides）。某些品种全白。""",
    """A. phalloides，又名死亡帽，是已知所有蘑菇中最有毒的一种。""",
]

对于这个例子，我们将创建一个小数据库，我们可以作为一个示例来使用。

In [28]:
smalldb.delete_collection()
smalldb = Chroma.from_texts(texts, embedding=embedding)


In [29]:
len(smalldb)

3

In [13]:
smalldb_chinese = Chroma.from_texts(texts_chinese, embedding=embedding)

下面是我们对于这个示例所提出的问题

In [30]:
question = "Tell me about all-white mushrooms with large fruiting bodies"

In [15]:
question_chinese = "告诉我关于具有大型子实体的全白色蘑菇的信息"

现在，我们可以运行一个相似性搜索，设置k=2，只返回两个最相关的文档。

我们可以看到，没有提到它是有毒的事实。

In [34]:
smalldb.similarity_search(question, k=2)

[Document(page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.'),
 Document(page_content='A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.')]

In [21]:
smalldb_chinese.similarity_search(question_chinese, k=2)

[Document(page_content='一种具有大型子实体的蘑菇是毒鹅膏菌（Amanita phalloides）。某些品种全白。'),
 Document(page_content='毒鹅膏菌（Amanita phalloides）具有大型且引人注目的地上（epigeous）子实体（basidiocarp）')]

现在，让我们运行最大边际相关性(MMR)。

设置k=2，因为我们仍然希望返回两个文档。设置fetch_k=3，fetch_k是我们最初获取的所有文档(3个)。

In [33]:
smalldb.max_marginal_relevance_search(question,k=2, fetch_k=3)

[Document(page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.'),
 Document(page_content='A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.')]

In [23]:
smalldb_chinese.max_marginal_relevance_search(question_chinese,k=2, fetch_k=3)

[Document(page_content='一种具有大型子实体的蘑菇是毒鹅膏菌（Amanita phalloides）。某些品种全白。'),
 Document(page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.')]

我们现在可以看到，我们检索的文档中返回了有毒的信息。

### 1.2 解决多样性：最大边际相关性(MMR)


我们刚刚通过一个示例引出了一个问题：如何加强搜索结果的多样性。
 
最大边际相关性(`Maximum marginal relevance`)试图在查询的相关性和结果的多样性之间实现两全其美。

让我们回到上一节课的一个例子，当我们通过问题对向量数据库进行相似性搜索后

我们可以看看前两个文档，只看前几个字符，可以看到它们是相同的。

In [35]:
question = "what did they say about matlab?"
docs_ss = vectordb.similarity_search(question,k=3)

In [36]:
docs_ss[0].page_content[:100]

"Okay, and using this matrix vector notati on, I think, I don't know, I think we did this \nwhole thin"

In [37]:
docs_ss[1].page_content[:100]

'[End of Audio]  \nDuration: 69 minutes'

In [38]:
question_chinese = "Matplotlib是什么？"
docs_ss_chinese = vectordb_chinese.similarity_search(question_chinese,k=3)

In [39]:
docs_ss_chinese[0].page_content[:100]

'第⼀回：Matplotlib 初相识\n⼀、认识matplotlib\nMatplotlib 是⼀个 Python 2D 绘图库，能够以多种硬拷⻉格式和跨平台的交互式环境⽣成出版物质量的图形，⽤来绘制各种'

In [40]:
docs_ss_chinese[1].page_content[:100]

'第⼀回：Matplotlib 初相识\n⼀、认识matplotlib\nMatplotlib 是⼀个 Python 2D 绘图库，能够以多种硬拷⻉格式和跨平台的交互式环境⽣成出版物质量的图形，⽤来绘制各种'

注意：使用`MMR`所得出结果的差异。

In [55]:
docs_mmr = vectordb.max_marginal_relevance_search(question,k=3)

In [56]:
docs_mmr_chinese = vectordb_chinese.max_marginal_relevance_search(question_chinese,k=3)

当我们运行MMR后得到结果时，我们可以看到第一个与之前的相同，因为那是最相似的。

In [57]:
docs_mmr[0].page_content[:100]

"Okay, and using this matrix vector notati on, I think, I don't know, I think we did this \nwhole thin"

In [58]:
docs_mmr_chinese[0].page_content[:100]

'第⼀回：Matplotlib 初相识\n⼀、认识matplotlib\nMatplotlib 是⼀个 Python 2D 绘图库，能够以多种硬拷⻉格式和跨平台的交互式环境⽣成出版物质量的图形，⽤来绘制各种'

In [60]:
docs_mmr_chinese

[Document(page_content='第⼀回：Matplotlib 初相识\n⼀、认识matplotlib\nMatplotlib 是⼀个 Python 2D 绘图库，能够以多种硬拷⻉格式和跨平台的交互式环境⽣成出版物质量的图形，⽤来绘制各种静态，动态，\n交互式的图表。\nMatplotlib 可⽤于 Python 脚本， Python 和 IPython Shell 、 Jupyter notebook ， Web 应⽤程序服务器和各种图形⽤户界⾯⼯具包等。\nMatplotlib 是 Python 数据可视化库中的泰⽃，它已经成为 python 中公认的数据可视化⼯具，我们所熟知的 pandas 和 seaborn 的绘图接⼝\n其实也是基于 matplotlib 所作的⾼级封装。\n为了对matplotlib 有更好的理解，让我们从⼀些最基本的概念开始认识它，再逐渐过渡到⼀些⾼级技巧中。\n⼆、⼀个最简单的绘图例⼦\nMatplotlib 的图像是画在 figure （如 windows ， jupyter 窗体）上的，每⼀个 figure ⼜包含了⼀个或多个 axes （⼀个可以指定坐标系的⼦区\n域）。最简单的创建 figure 以及 axes 的⽅式是通过 pyplot.subplots命令，创建 axes 以后，可以使⽤ Axes.plot绘制最简易的折线图。\nimport matplotlib.pyplot as plt\nimport matplotlib as mpl\nimport numpy as np\nfig, ax = plt.subplots()  # 创建⼀个包含⼀个 axes 的 figure\nax.plot([1, 2, 3, 4], [1, 4, 2, 3]);  # 绘制图像\nTrick： 在jupyter notebook 中使⽤ matplotlib 时会发现，代码运⾏后⾃动打印出类似 <matplotlib.lines.Line2D at 0x23155916dc0>\n这样⼀段话，这是因为 matplotlib 的绘图代码默认打印出最后⼀个对象。如果不想显示这句话，有以下三种⽅法，在本章节的代码示例\n中你能找到这三种⽅法的使⽤。\n\x00. 在代码块最后加⼀个分号 ;\n\x00. 在代码块最后

但是当我们进行到第二个时，我们可以看到它是不同的。

它在回应中获得了一些多样性。

In [30]:
docs_mmr[1].page_content[:100]

'algorithm then? So what’s different? How come  I was making all that noise earlier about \nleast squa'

In [59]:
docs_mmr_chinese[1].page_content[:100]

'Axes helper method Artist Container\nbar - bar charts Rectangle ax.patches\nerrorbar - error bar plots'

### 1.3 解决特殊性：使用元数据


在上一节课中，我们展示了一个问题，是询问了关于文档中某一讲的问题，但得到的结果中也包括了来自其他讲的结果。

为了解决这一问题，很多向量数据库都支持对`metadata`的操作。

`metadata`为每个嵌入的块(embedded chunk)提供上下文。

In [61]:
question = "what did they say about regression in the third lecture?"

In [62]:
question_chinese = "他们在第二讲中对Figure说了些什么？"  

现在，我们以手动的方式来解决这个问题，我们会指定一个元数据过滤器`filter`

In [63]:
docs = vectordb.similarity_search(
    question,
    k=3,
    filter={"source":"docs/cs229_lectures/MachineLearning-Lecture03.pdf"}
)

In [64]:
docs_chinese = vectordb_chinese.similarity_search(
    question_chinese,
    k=3,
    filter={"source":"docs/matplotlib/第二回：艺术画笔见乾坤.pdf"}
)

接下来，我们可以看到结果都来自对应的章节

In [65]:
for d in docs:
    print(d.metadata)

{'page': 9, 'source': 'docs/cs229_lectures/MachineLearning-Lecture03.pdf'}
{'page': 5, 'source': 'docs/cs229_lectures/MachineLearning-Lecture03.pdf'}
{'page': 15, 'source': 'docs/cs229_lectures/MachineLearning-Lecture03.pdf'}


In [66]:
for d in docs_chinese:
    print(d.metadata)
    

{'page': 12, 'source': 'docs/matplotlib/第二回：艺术画笔见乾坤.pdf'}
{'page': 11, 'source': 'docs/matplotlib/第二回：艺术画笔见乾坤.pdf'}
{'page': 2, 'source': 'docs/matplotlib/第二回：艺术画笔见乾坤.pdf'}


当然，我们不能每次都采用手动的方式来解决这个问题，这会显得不够智能

下一小节将要展示通过LLM来解决这个问题

### 1.4 解决特殊性：在元数据中使用自查询检索器

我们有一个有趣的挑战：我们通常希望从查询本身来推断元数据。

为了解决这个问题，我们可以使用SelfQueryRetriever，它使用LLM来提取：
 
1. 用于向量搜索的查询(`query`)字符串，即：问题
2. 要一起传入的元数据过滤器

大多数向量数据库支持元数据过滤器，因此不需要任何新的数据库及索引。

In [67]:
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

In [39]:
llm = OpenAI(temperature=0)

`AttributeInfo`是我们可以指定元数据中的不同字段以及它们对应的位置。

在元数据中，我们只有两个字段，`source`和`page`。

我们将填写每个属性的名称、描述和类型的描述。

这些信息实际上将被传递给LLM，所以需要尽可能详细地描述。

In [68]:
metadata_field_info = [
    AttributeInfo(
        name="source",
        description="The lecture the chunk is from, should be one of `docs/cs229_lectures/MachineLearning-Lecture01.pdf`, `docs/cs229_lectures/MachineLearning-Lecture02.pdf`, or `docs/cs229_lectures/MachineLearning-Lecture03.pdf`",
        type="string",
    ),
    AttributeInfo(
        name="page",
        description="The page from the lecture",
        type="integer",
    ),
]

In [71]:
metadata_field_info_chinese = [
    AttributeInfo(
        name="source",
        description="讲义来源于 `docs/matplotlib/第一回：Matplotlib初相识.pdf`, `docs/matplotlib/第二回：艺术画笔见乾坤.pdf`, or `docs/matplotlib/第三回：布局格式定方圆.pdf` 的其中之一",
        type="string",
    ),
    AttributeInfo(
        name="page",
        description="讲义的那一页",
        type="integer",
    ),
]

In [72]:
document_content_description = "Lecture notes"
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectordb,
    document_content_description,
    metadata_field_info,
    verbose=True
)

In [73]:
document_content_description_chinese = "课堂讲义"
retriever_chinese = SelfQueryRetriever.from_llm(
    llm,
    vectordb_chinese,
    document_content_description_chinese,
    metadata_field_info_chinese,
    verbose=True
)

In [74]:
question = "what did they say about regression in the third lecture?"

In [92]:
question_chinese = "他们在第二讲中对Figure说了些什么？"  

当你第一次执行下一行时，你会收到关于predict_and_parse已被弃用的**警告**。 这可以安全地忽略。

In [76]:
docs = retriever.get_relevant_documents(question)

/root/miniconda3/envs/python3.9/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [93]:
docs_chinese = retriever_chinese.get_relevant_documents(question_chinese)

In [78]:
for d in docs:
    print(d.metadata)

{'page': 6, 'source': 'docs/cs229_lectures/MachineLearning-Lecture03.pdf'}
{'page': 0, 'source': 'docs/cs229_lectures/MachineLearning-Lecture03.pdf'}
{'page': 2, 'source': 'docs/cs229_lectures/MachineLearning-Lecture03.pdf'}
{'page': 10, 'source': 'docs/cs229_lectures/MachineLearning-Lecture03.pdf'}


In [94]:
for d in docs_chinese:
    print(d.metadata)

{'page': 2, 'source': 'docs/matplotlib/第三回：布局格式定方圆.pdf'}
{'page': 2, 'source': 'docs/matplotlib/第一回：Matplotlib初相识.pdf'}
{'page': 2, 'source': 'docs/matplotlib/第一回：Matplotlib初相识.pdf'}
{'page': 2, 'source': 'docs/matplotlib/第二回：艺术画笔见乾坤.pdf'}


### 1.5 其他技巧：压缩

另一种提高检索到的文档质量的方法是压缩。

与查询最相关的信息可能隐藏在具有大量不相关文本的文档中。

在应用程序中传递完整的文档可能会导致更昂贵的LLM调用和更差的响应。

上下文压缩就是为了解决这个问题。

In [95]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [96]:
def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))

In [97]:
# llm = OpenAI(temperature=0)
compressor = LLMChainExtractor.from_llm(llm)  # 压缩器

In [98]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever()
)

In [99]:
compression_retriever_chinese = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb_chinese.as_retriever()
)

In [100]:
question = "what did they say about matlab?"
compressed_docs = compression_retriever.get_relevant_documents(question)
pretty_print_docs(compressed_docs)

Document 1:

"we couldn't have if I was writing out reams of"


In [101]:
question_chinese = "Matplotlib是什么？"
compressed_docs_chinese = compression_retriever_chinese.get_relevant_documents(question_chinese)
pretty_print_docs(compressed_docs_chinese)

Document 1:

Matplotlib是一个Python 2D绘图库，能够以多种硬拷贝格式和跨平台的交互式环境生成出版物质量的图形，用于绘制各种静态，动态，交互式的图表。它是Python数据可视化库中的佼佼者，pandas和seaborn的绘图接口基于matplotlib的高级封装。最简单的创建figure以及axes的方式是通过pyplot.subplots命令，然后使用Axes.plot绘制图形。
----------------------------------------------------------------------------------------------------
Document 2:

Matplotlib是一个Python 2D绘图库，能够以多种硬拷贝格式和跨平台的交互式环境生成出版物质量的图形，用于绘制各种静态，动态，交互式的图表。它是Python数据可视化库中的佼佼者，pandas和seaborn的绘图接口基于matplotlib的高级封装。最简单的创建figure以及axes的方式是通过pyplot.subplots命令，然后可以使用Axes.plot绘制图形。
----------------------------------------------------------------------------------------------------
Document 3:

matplotlib 是一个用于数据可视化的开源小組，它提供了丰富的 API 进行绘图。在实际使用中，用户可能会使用 `plot()` 函数绘制线图，如 `plt.plot(x, x, label='linear')`，并通过 `xlabel()`, `ylabel()`, 和 `title()` 设置轴标签和图表标题。matplotlib 还支持创建更复杂的图表，通过定义布局（例如，`subplots()`），设置样式，以及添加图例等。OO 模式（Object-Oriented）和 pyplot 模式是 matplotlib 的两种绘图模式，OO 模式以对象的形式组织图形，而 pyplot 模式则更加简洁，通常通过函数调用来完成绘图。
----------------------------------------------

现在当我们提出问题后，查看结果文档

我们可以看到两件事。

1. 它们比正常文档短很多
2. 仍然有一些重复的东西，这是因为在底层我们使用的是语义搜索算法。

这就是我们在本课程前面使用MMR解决的问题。

这是一个很好的例子，你可以结合各种技术得到最好的可能结果。

## 二、结合各种技术

为了做到这一点，我们在从向量数据库创建检索器时，可以将搜索类型设置为MMR。

然后我们可以重新运行这个过程，看到我们返回的是一个过滤过的结果集，其中不包含任何重复的信息。

In [102]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever(search_type = "mmr")
)

In [103]:
compression_retriever_chinese = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb_chinese.as_retriever(search_type = "mmr")
)

In [104]:
question = "what did they say about matlab?"
compressed_docs = compression_retriever.get_relevant_documents(question)
pretty_print_docs(compressed_docs)

Document 1:

"we couldn't have if I was writing out reams of"


In [105]:
question_chinese = "Matplotlib是什么？"
compressed_docs_chinese = compression_retriever_chinese.get_relevant_documents(question_chinese)
pretty_print_docs(compressed_docs_chinese)

Document 1:

Matplotlib是一个Python 2D绘图库，能够以多种硬拷贝格式和跨平台的交互式环境生成出版物质量的图形，用于绘制各种静态，动态，交互式的图表。它是Python数据可视化库中的佼佼者，pandas和seaborn的绘图接口基于matplotlib的高级封装。最简单的创建figure以及axes的方式是通过pyplot.subplots命令，然后可以使用Axes.plot绘制图形。
----------------------------------------------------------------------------------------------------
Document 2:

matplotlib.patches.Patch 类是二维图形类...本小节重点讲述三种最常见的子类，矩形，多边形和楔形...Rectangle矩形类在官网中的定义是...在实际中最常见的矩形图是 hist直方图和bar条形图...hist-直方图...matplotlib.pyplot.hist(x,bins=None,range=None, density=None, bottom=None, histtype='bar', align='mid', log=False, color=None, label=None, stacked=False, normed=None)...
----------------------------------------------------------------------------------------------------
Document 3:

matplotlib 中曲线的绘制，主要是通过类 matplotlib.lines.Line2D 来完成的。
matplotlib 中线-line的含义：它表示的可以是连接所有顶点的实线样式，也可以是每个顶点的标记。此外，这条线也会受到绘画风格的影响，比如，我们可以创建虚线种类的线。
它的构造函数:
class matplotlib.lines.Line2D(xdata, ydata, linewidth=None, linestyle=None, color=None, marker=None,
markersize=None

## 三、其他类型的检索

值得注意的是，vetordb并不是唯一一种检索文档的工具。

`LangChain`检索器抽象包括其他检索文档的方式，如：`TF-IDF` 或 `SVM`。

In [106]:
from langchain.retrievers import SVMRetriever
from langchain.retrievers import TFIDFRetriever
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [107]:
# 加载PDF
loader = PyPDFLoader("docs/cs229_lectures/MachineLearning-Lecture01.pdf")
pages = loader.load()
all_page_text = [p.page_content for p in pages]
joined_page_text = " ".join(all_page_text)

# 分割文本
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1500,chunk_overlap = 150)
splits = text_splitter.split_text(joined_page_text)


In [108]:
# 加载PDF
loader_chinese = PyPDFLoader("docs/matplotlib/第一回：Matplotlib初相识.pdf")
pages_chinese = loader_chinese.load()
all_page_text_chinese = [p.page_content for p in pages_chinese]
joined_page_text_chinese = " ".join(all_page_text_chinese)

# 分割文本
text_splitter_chinese = RecursiveCharacterTextSplitter(chunk_size = 1500,chunk_overlap = 150)
splits_chinese = text_splitter_chinese.split_text(joined_page_text_chinese)

In [115]:
# 检索
# svm_retriever = SVMRetriever.from_texts(splits, embedding)
tfidf_retriever = TFIDFRetriever.from_texts(splits)

In [110]:
question = "What are major topics for this class?"  # 这门课的主要主题是什么？
docs_svm = svm_retriever.get_relevant_documents(question)
docs_svm[0]

NameError: name 'svm_retriever' is not defined

In [116]:
question = "what did they say about matlab?"  # 他们关于Matlab说了些什么？
docs_tfidf = tfidf_retriever.get_relevant_documents(question)
docs_tfidf[0]

Document(page_content="Saxena and Min Sun here did, wh ich is given an image like this, right? This is actually a \npicture taken of the Stanford campus. You can apply that sort of cl ustering algorithm and \ngroup the picture into regions. Let me actually blow that up so that you can see it more \nclearly. Okay. So in the middle, you see the lines sort of groupi ng the image together, \ngrouping the image into [inaudible] regions.  \nAnd what Ashutosh and Min did was they then  applied the learning algorithm to say can \nwe take this clustering and us e it to build a 3D model of the world? And so using the \nclustering, they then had a lear ning algorithm try to learn what the 3D structure of the \nworld looks like so that they could come up with a 3D model that you can sort of fly \nthrough, okay? Although many people used to th ink it's not possible to take a single \nimage and build a 3D model, but using a lear ning algorithm and that sort of clustering \nalgorithm is the first ste

In [117]:
# 中文版
# svm_retriever_chinese = SVMRetriever.from_texts(splits_chinese, embedding)
tfidf_retriever_chinese = TFIDFRetriever.from_texts(splits_chinese)

In [118]:
question_chinese = "这门课的主要主题是什么？" 
docs_svm_chinese = svm_retriever_chinese.get_relevant_documents(question_chinese)
docs_svm_chinese[0]

NameError: name 'svm_retriever_chinese' is not defined

In [119]:
question_chinese = "Matplotlib是什么？"
docs_tfidf_chinese = tfidf_retriever_chinese.get_relevant_documents(question_chinese)
docs_tfidf_chinese[0]

Document(page_content='fig, ax = plt.subplots()  \n# step4 绘制图像，  这⼀模块的扩展参考第⼆章进⼀步学习\nax.plot(x, y, label=\'linear\')  \n# step5 添加标签，⽂字和图例，这⼀模块的扩展参考第四章进⼀步学习\nax.set_xlabel(\'x label\') \nax.set_ylabel(\'y label\') \nax.set_title("Simple Plot")  \nax.legend() ;\n思考题\n请思考两种绘图模式的优缺点和各⾃适合的使⽤场景\n在第五节绘图模板中我们是以 OO 模式作为例⼦展示的，请思考并写⼀个 pyplot 绘图模式的简单模板')